In [32]:
"""
experimentation.ipynb

File for experimenting with new things

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity import keras as sparsity

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
# from src.harness.experiment import experiment, ExperimentData
from src.harness.model import create_model, LeNet300, load_model
from src.harness.pruning import prune_by_percent
from src.harness.training import TrainingRound
from src.lottery_ticket.foundations import paths

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
# Constants
DEBUG: bool = True

In [39]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
Y_test = tf.convert_to_tensor(Y_test, dtype=tf.float32)

# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
def create_lenet_300_100(random_seed: int, input_shape: tuple[int, ...], num_classes: int) -> keras.Model:
    """
    Simple hardcoded class definition for creating the sequential Keras equivalent to LeNet-300-100.
    """

    # Set seeds for reproducability
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = keras.Sequential(name="LeNet-300-100")
    model.add(keras.layers.Flatten(input_shape=input_shape))
    model.add(keras.layers.Dense(300, activation='relu'))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    return model

def pruned_nn(
        random_seed: int, 
        input_shape: tuple[int, ...], 
        num_classes: int, 
        pruning_params: dict, 
        loss: keras.losses.Loss = keras.losses.categorical_crossentropy, 
        optimizer=C.OPTIMIZER()) -> keras.Model:
    """
    Function to define the architecture of a neural network model
    following 300 100 architecture for MNIST dataset and using
    provided parameter which are used to prune the model.
    
    Input: 'pruning_params' Python 3 dictionary containing parameters which are used for pruning
    Output: Returns designed and compiled neural network model
    """
    # Set seeds for reproducability
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = sparsity.prune_low_magnitude(keras.Sequential([
        keras.layers.Flatten(input_shape=input_shape),
        keras.layers.Dense(units = 300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        # model.add(l.Dropout(0.2))
        keras.layers.Dense(units = 100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        # model.add(l.Dropout(0.1))
        keras.layers.Dense(units = num_classes, activation='softmax')
    ]), **pruning_params)
    
    # Compile pruned CNN-
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

def create_masked_nn(*args) -> keras.Model:
    """
    Create a masked neural network where all the weights are initialized to 1s.
    """
    model: keras.Model = pruned_nn(*args)
    model_stripped = sparsity.strip_pruning(model)
    # Assign all weights to 1 to start
    for weights in model_stripped.trainable_weights:
        weights.assign(
            tf.ones_like(
                input = weights,
                dtype = tf.float32
            )
        )
    return model

In [13]:
# Define pruning parameters and callback
def create_pruning_parameters(target_sparsity: float, begin_Step: int, end_step: int, frequency: int) -> dict:
    """
    Create the dictionary of pruning parameters to be used.
    """
    return {
        'pruning_schedule': sparsity.ConstantSparsity(
            target_sparsity=target_sparsity, 
            begin_step=begin_Step,
            end_step=end_step, 
            frequency=frequency
        )
    }

def create_pruning_callback(monitor: str, patience: int, minimum_delta: float) -> list:
    """
    Create a callback to be performed during pruning.
    """
    return [
        sparsity.UpdatePruningStep(),
        # sparsity.PruningSummaries(log_dir = logdir, profile_batch=0),
        tf.keras.callbacks.EarlyStopping(
            monitor=monitor, 
            patience=patience,
            min_delta=minimum_delta
        )
    ]

def create_pruning_percentages():
    pass

pruning_params_unpruned: dict = create_pruning_parameters(0.01, 0, 0, 100)
pruning_callback: list = create_pruning_callback('val_loss', 3, 0.001)

In [14]:
# For each layer, there are synaptic connections from the previous layer and the neurons
def get_layer_weight_counts(model: keras.Model) -> list[int]:
    """
    Function to return a list of integer values for the number of 
    parameters in each layer.
    """
    def get_num_layer_weights(layer: keras.layers.Layer) -> int:
        layer_weight_count: int = 0
        weights: list[np.array] = layer.get_weights()

        for idx in range(len(weights))[::2]:
            synapses: np.ndarray = weights[idx]
            neurons: np.array = weights[idx + 1]
            layer_weight_count += np.prod(synapses.shape) + np.prod(neurons.shape)

        return layer_weight_count
    
    return list(map(get_num_layer_weights, model.layers))

def get_pruning_percents(
        layer_weight_counts: list[int], 
        first_step_pruning_percent: float,
        target_sparsity: float
        ) -> list[np.array]:
    """
    Function to get arrays of model sparsity at each step of pruning.
    """

    def total_sparsity(
            original_weight_counts: list[int], 
            current_weight_counts: list[int]
            ) -> float:
        """
        Helper function to calculate total sparsity of parameters.
        """
        return np.sum(current_weight_counts) / np.sum(original_weight_counts)
    
    def sparsify(
            original_weight_counts: list[int], 
            current_weight_counts: list[int], 
            original_pruning_percent: float
            ) -> list[float]:
        sparsities: list[float] = []
        for idx, (original, current) in enumerate(zip(original_weight_counts, current_weight_counts)):
            if original == 0:
                continue
            new_weight_count: int = np.round(current * (1 - original_pruning_percent))
            sparsities.append((original - new_weight_count) / original)
            current_weight_counts[idx] = new_weight_count
        return np.round(np.mean(sparsities), decimals=5)
    
    sparsities: list[float] = []
    
    # Elementwise copy
    current_weight_counts: list[int] = [weight_count for weight_count in layer_weight_counts]
    
    while total_sparsity(layer_weight_counts, current_weight_counts) > target_sparsity:
        sparsities.append(sparsify(layer_weight_counts, current_weight_counts, first_step_pruning_percent))

    return sparsities

In [15]:
def count_nonzero_parameters(model: keras.Model):
    """
    Print summary for the number of nonzero parameters in the model.
    """
    model_sum_params: int = 0
    model_stripped: keras.Model = sparsity.strip_pruning(model)
    weights: list[np.ndarray] = model_stripped.trainable_weights
    for idx in range(len(weights))[::2]:
        layer_number: int = int(idx / 2)
        synapses: np.ndarray = weights[idx]
        nonzero_synapses: int = tf.math.count_nonzero(synapses, axis=None).numpy()
        neurons: np.array = weights[idx + 1]
        nonzero_neurons: int = tf.math.count_nonzero(neurons, axis=None).numpy()

        if DEBUG:
            print(f'Nonzero parameters in layer {layer_number} synapses:', nonzero_synapses)
            print(f'Nonzero parameters in layer {layer_number} neurons:', nonzero_neurons)
        
        model_sum_params += nonzero_synapses + nonzero_neurons
    
    if DEBUG:
        print(f'Total nonzero parameters: {model_sum_params}')
    

In [16]:
def create_metrics() -> tuple[tf.keras.metrics.Metric, ...]:
    """
    Create metrics to measure the error and accuracy of the model.
    """
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

    return train_loss, train_accuracy, test_loss, test_accuracy

train_loss, train_accuracy, test_loss, test_accuracy = create_metrics()

In [17]:
# Prepare MNIST for training
X_train, Y_train, X_test, Y_test = load_and_process_mnist()
train_dataset: tuple[np.array, np.array] = (X_train, Y_train)
test_dataset: tuple[np.array, np.array] = (X_test, Y_test)

In [40]:
# Create a masked model
args: tuple = (0, X_train[0].shape, 10, pruning_params_unpruned)
model = sparsity.strip_pruning(pruned_nn(*args))
mask_model = sparsity.strip_pruning(create_masked_nn(*args))
layer_weight_counts: list[int] = get_layer_weight_counts(mask_model)
sparsities: list[float] = get_pruning_percents(layer_weight_counts, .2, .01)
num_pruning_rounds: int = len(sparsities)
count_nonzero_parameters(mask_model)

/Users/jordan/anaconda3/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: `prune_low_magnitude` can only prune an object of the following types: keras.models.Sequential, keras functional model, keras.layers.Layer, list of keras.layers.Layer. You passed an object of type: Sequential.

In [ ]:
@tf.function
def train_one_step(model: keras.Model,
                   mask_model: keras.Model, 
                   inputs: tf.Tensor, 
                   labels: tf.Tensor,
                   loss_fn: callable = keras.losses.CategoricalCrossentropy(), 
                   optimizer: keras.optimizers.Optimizer = C.OPTIMIZER(), 
                   ):
    """
    Function to compute one step of gradient descent optimization
    """

    with tf.GradientTape() as tape:
        # Make predictions using defined model-
        y_pred = model(inputs)

        # Compute loss
        loss = loss_fn(labels, y_pred)

    # Compute gradients with respect to defined loss and weights and biases
    gradients = tape.gradient(loss, model.trainable_variables)

    # List to hold element-wise multiplication between
    # computed gradient and masks
    grad_mask_mul = []

    # Perform element-wise multiplication between computed gradients and masks
    for grad_layer, mask in zip(gradients, mask_model.trainable_weights):
        grad_mask_mul.append(tf.math.multiply(grad_layer, mask))

    # Apply computed gradients to model's weights and biases
    optimizer.apply_gradients(zip(grad_mask_mul, model.trainable_variables))

    # Compute accuracy
    return train_loss(loss), train_accuracy(labels, y_pred)

@tf.function
def test_step(model: keras.Model, 
              data: tf.Tensor, 
              labels: tf.Tensor,
              loss_fn: callable = keras.losses.CategoricalCrossentropy(),
              ):
    """
    Function to test model performance on testing dataset
    """
    predictions = model(data)
    return loss_fn(labels, predictions), test_accuracy(predictions, labels)

In [ ]:
def make_args() -> tuple:
    args: tuple = (0, X_train[0].shape, 10, pruning_params_unpruned)
    model = sparsity.strip_pruning(pruned_nn(*args))
    mask_model = sparsity.strip_pruning(create_masked_nn(*args))
    return model, mask_model, X_train, Y_train

# args: tuple = make_args()
# test_loss, test_accuracy = test_step(model, X_train, Y_train)
# print(f'Untrained Test: Loss: {test_loss.numpy()}, Accuracy: {test_accuracy.numpy()}')

# for i in range(100):
#     training_loss, training_accuracy = train_one_step(*args)
#     print(f'Training iteration {i + 1}: Loss: {training_loss.numpy()}, Accuracy: {training_accuracy.numpy()}')
print(args[0].trainable_weights)
test_loss, test_accuracy = test_step(model, X_train, Y_train)
print(f'Trained Test: Loss: {test_loss.numpy()}, Accuracy: {test_accuracy.numpy()}')

[<tf.Variable 'prune_low_magnitude_dense_432/kernel:0' shape=(784, 300) dtype=float32, numpy=
array([[ 0.00233377, -0.02841079,  0.05042759, ...,  0.04471111,
         0.0469005 , -0.00633286],
       [ 0.04599051,  0.07108735, -0.06869254, ...,  0.0193451 ,
        -0.03234061,  0.03169315],
       [ 0.04332277,  0.00188898, -0.05098354, ...,  0.05931461,
         0.0038614 , -0.06489589],
       ...,
       [ 0.06133612, -0.00425664, -0.03556142, ...,  0.05923072,
        -0.04446548,  0.03342723],
       [ 0.02262612, -0.01283204,  0.03786094, ..., -0.03319833,
         0.0259892 , -0.02313284],
       [ 0.00880335,  0.00193495, -0.07007851, ..., -0.05422467,
        -0.0134997 , -0.03317983]], dtype=float32)>, <tf.Variable 'prune_low_magnitude_dense_432/bias:0' shape=(300,) dtype=float32, numpy=
array([ 1.32970903e-02, -1.56251859e-04,  8.03746004e-03, -9.55416169e-03,
        8.31857696e-03,  8.97290185e-03,  1.79232322e-02,  9.47246980e-03,
        2.53446326e-02, -1.38653256e-03

In [ ]:
# Training function

def save_model(model: keras.Model, seed: int, pruning_step: int, untrained: bool = False,):
    """
    Function to save a single trained model.

    :param model:        Model object being saved.
    :param seed:         Random seed used in the model
    :param pruning_step: Integer value for the number of pruning steps which had been completed for the model.
    :param untrained:    Boolean for if it is the untrained version of a model. Defaults to False.
    """

    output_directory: str = paths.get_model_directory(seed, C.MODEL_DIRECTORY)
    # Save the initial weights in an 'initial' directory in the top-level of the model directory
    if untrained:
        untrained_directory: str = paths.initial(output_directory)
        model.save_weights(paths.weights(untrained_directory))
    else:
        # Create a trial directory within the model directory
        trial_directory: str = paths.trial(output_directory, pruning_step)
        model.save_weights(paths.weights(trial_directory))

def training_loop(
        pruning_step: int,
        model_stripped: keras.Model, 
        mask_model_stripped: keras.Model,
        make_dataset: callable,
        num_epochs: int, 
        patience: int,
        minimum_delta: float,
        optimizer: keras.optimizers.Optimizer = C.OPTIMIZER(),
    ) -> keras.Model:
    """
    Main training loop for the model.
    """
    # Number of epochs without improvement
    local_patience: int = 0
    best_test_loss: float = float('inf')

    # Extract input and target
    X_train, Y_train, X_test, Y_test = make_dataset()

    initial_weights: list[np.ndarray] = model_stripped.trainable_weights
    masks: list[np.ndarray] = mask_model_stripped.trainable_weights
    train_losses: np.array = np.zeros(Y_train.shape[0])
    train_accuracies: np.array = np.zeros(Y_train.shape[0])
    test_losses: np.array = np.zeros(Y_test.shape[0])
    test_accuracies: np.array = np.zeros(Y_test.shape[0])

    for epoch in range(num_epochs):
        
        # Exit early if there are `patience` epochs without improvement
        if local_patience >= patience:
            if DEBUG:
                print(f'Early stopping initiated')
            break
        
        # Update model parameters for each point in the training set
        for x, y in X_train, Y_train:
            train_loss, train_accuracy = train_one_step(model_stripped, mask_model_stripped, optimizer, x, y)
            train_losses[idx] = train_loss
            train_accuracies[idx] = train_accuracy

        # Evaluate model on each point in the test set
        for idx, (x_t, y_t) in enumerate(zip(X_test, Y_test)):
            test_loss, test_accuracy = test_step(model_stripped, optimizer, x_t, y_t)
            test_losses[idx] = test_loss
            test_accuracies[idx] = test_accuracy

        # Display output
        if DEBUG:
            print(f'Epoch {epoch + 1}, Train/Test Loss: {train_loss:.4f}/{test_loss:.4f}, Train/Test Accuracy: {train_accuracy:.4f}/{test_accuracy:.4f}')
            print(f'Total number of trainable parameters = {np.sum(count_nonzero_parameters(model_stripped))}')

        # Check for early stopping criteria
        mean_test_loss: float = np.mean(test_losses)
        if mean_test_loss < best_test_loss and (best_test_loss - mean_test_loss) >= minimum_delta:
            # update 'best_test_loss' variable to lowest loss encountered so far
            best_test_loss = mean_test_loss
            # Reset the counter
            local_patience = 0
        else:  # there is no improvement in monitored metric 'val_loss'
            local_patience += 1  # number of epochs without any improvement

    final_weights: list[np.ndarray] = model_stripped.trainable_weights

    # Compile training round data
    round_data: TrainingRound = TrainingRound(pruning_step, initial_weights, final_weights, masks, train_losses, train_accuracies, test_losses, test_accuracies)

    return model_stripped, round_data
    

def train(
    pruning_step: int,
    model: keras.Model, 
    mask_model: keras.Model,
    make_dataset: callable, 
    num_epochs: int = C.TRAINING_EPOCHS,
    patience: int = C.PATIENCE,
    minimum_delta: float = C.MINIMUM_DELTA,
    optimizer: tf.keras.optimizers.Optimizer = C.OPTIMIZER(), 
    ) -> tuple[keras.Model, keras.Model, TrainingRound]:
    """
    Function to perform training for a model.

    :param pruning_step:     Integer value for the step in pruning. Defaults to 0.
    :param model:            Model to optimize.
    :param mask_model:       Model whose weights correspond to masks being applied.

    :param make_dataset:     Function to produce the training/test sets.
    :param num_epochs:       Number of epochs to train for.
    :param patience:         Number of epochs which can be ran without improvement before calling early stopping.
    :param minimum_delta:    Minimum increase to be considered an improvement.s
    :param optimizer:        Optimizer to use during training.

    :returns: Model, masked model, and training round objects with the final trained model and the training summary/.
    """

    if pruning_step == 0:
        save_model(model, pruning_step, untrained=True)

    # Run the training loop
    model, training_round = training_loop(
        pruning_step, 
        sparsity.strip_pruning(model), 
        sparsity.strip_pruning(mask_model), 
        make_dataset, 
        num_epochs, 
        patience, 
        minimum_delta, 
        optimizer
    )

    # Save network final weights and masks to its folder in the appropriate trial folder
    save_model(model, pruning_step)

    return model, mask_model, training_round

AttributeError: module 'src.harness.constants' has no attribute 'TRAINING_EPOCHS'

In [ ]:
# Training
# https://github.com/arjun-majumdar/Lottery_Ticket_Hypothesis-TensorFlow_2/blob/master/


def experiment(make_dataset: callable, 
               make_model: callable, 
               train_model: callable, 
               prune_masks: callable, 
               optimizer: keras.optimizers.Optimizer,
               pruning_steps: int,
               num_epochs: int,
               patience: float,
               minimum_delta: float,
 ) -> ExperimentData:
    summary: ExperimentData = ExperimentData()

    for i in range(1, pruning_steps + 1):
        
        print("\n\n\nIterative pruning round: {0}\n\n".format(i))
        
        # Define 'train_one_step()' and 'test_step()' functions here-

        # Instantiate a model
        model_gt = pruned_nn(pruning_params_unpruned)
        
        # Load winning ticket (from above)-
        # model_gt.load_weights("LeNet_MNIST_Winning_Ticket.h5")
        model_gt.set_weights(winning_ticket_model.get_weights())
        
        # Strip model of pruning parameters-
        model_gt_stripped = sparsity.strip_pruning(model_gt)
        
        
        # Train model using 'GradientTape'-
        
        # Initialize parameters for Early Stopping manual implementation-
        best_val_loss = 100
        loc_patience = 0
        
        for epoch in range(num_epochs):
        
            if loc_patience >= patience:
                print("\n'EarlyStopping' called!\n")
                break
            
            # Reset the metrics at the start of the next epoch
            train_loss.reset_states()
            train_accuracy.reset_states()
            test_loss.reset_states()
            test_accuracy.reset_states()
            
            
            for x, y in train_dataset:
                # train_one_step(model_gt_stripped, mask_model, optimizer, x, y, grad_mask_mul)
                train_one_step(model_gt_stripped, mask_model_stripped, optimizer, x, y)


            for x_t, y_t in test_dataset:
                # test_step(x_t, y_t)
                test_step(model_gt_stripped, optimizer, x_t, y_t)
        
            # 'i' is the index for number of pruning rounds-
            round: TrainingRound = TrainingRound(i, {}, {}, {}, {}, -1, -1, 
                                                train_accuracy.result(), 
                                                train_loss.result(), 
                                                test_accuracy.result(), 
                                                test_loss.result())
            summary.add_pruning_round(round)
        
            # Count number of non-zero parameters in each layer and in total-
            # print("layer-wise manner model, number of nonzero parameters in each layer are: \n")

            model_sum_params = 0
        
            for layer in model_gt_stripped.trainable_weights:
                # print(tf.math.count_nonzero(layer, axis = None).numpy())
                model_sum_params += tf.math.count_nonzero(layer, axis = None).numpy()
        
            print("Total number of trainable parameters = {0}\n".format(model_sum_params))

            # Code for manual Early Stopping:
            if np.abs(test_loss.result() < best_val_loss) >= minimum_delta:
                # update 'best_val_loss' variable to lowest loss encountered so far-
                best_val_loss = test_loss.result()
            
                # reset 'loc_patience' variable-
                loc_patience = 0
            
            else:  # there is no improvement in monitored metric 'val_loss'
                loc_patience += 1  # number of epochs without any improvement

        
        # Save trained model weights-
        model_gt.save_weights("LeNet_MNIST_Trained_Weights.h5", overwrite=True)

        # Prune trained model:
        
        # Specify the parameters to be used for layer-wise pruning, Fully-Connected layer pruning-
        pruning_params_fc = {
            'pruning_schedule': sparsity.ConstantSparsity(
                target_sparsity=dense1_pruning[i - 1], begin_step = 1000,
                end_step = end_step, frequency=100
            )
        }
        
        
        # Instantiate a Nueal Network model to be pruned using parameters from above-
        pruned_model = pruned_nn(pruning_params_fc)
        
        # Load weights from original trained and unpruned model-
        # pruned_model.load_weights("LeNet_MNIST_Trained_Weights.h5")
        pruned_model.set_weights(model_gt.get_weights())
        
        # Train pruned NN-
        history_pruned = pruned_model.fit(
            x = X_train, y = y_train,
            batch_size = batch_size,
            epochs = epochs,
            verbose = 1,
            callbacks = callback,
            validation_data = (X_test, y_test),
            shuffle = True
        )
        
        # Strip the pruning wrappers from pruned model-
        pruned_model_stripped = sparsity.strip_pruning(pruned_model)
        
        # print("\nIn pruned model, number of nonzero parameters in each layer are: \n")
        pruned_sum_params = 0
        
        for layer in pruned_model_stripped.trainable_weights:
            # print(tf.math.count_nonzero(layer, axis = None).numpy())
            pruned_sum_params += tf.math.count_nonzero(layer, axis = None).numpy()
        

        # Create a mask:
        
        # Instantiate a new neural network model for which, the mask is to be created,
        mask_model = pruned_nn(pruning_params_unpruned)
        
        # Load weights of PRUNED model-
        # mask_model.load_weights("LeNet_MNIST_Pruned_Weights.h5")
        mask_model.set_weights(pruned_model.get_weights())
        
        # Strip the model of its pruning parameters-
        mask_model_stripped = sparsity.strip_pruning(mask_model)
        
        # For each layer, for each weight which is 0, leave it, as is.
        # And for weights which survive the pruning,reinitialize it to ONE (1)-
        for wts in mask_model_stripped.trainable_weights:
            wts.assign(tf.where(tf.equal(wts, 0.), 0., 1.))

        
        # Extract Winning Ticket:
        
        # Instantiate a new neural network model for which, the weights are to be extracted-
        winning_ticket_model = pruned_nn(pruning_params_unpruned)
        
        # Load weights of PRUNED model-
        # winning_ticket_model.load_weights("LeNet_MNIST_Pruned_Weights.h5")
        winning_ticket_model.set_weights(pruned_model.get_weights())
        
        # Strip the model of its pruning parameters-
        winning_ticket_model_stripped = sparsity.strip_pruning(winning_ticket_model)
        
        # For each layer, for each weight which is 0, leave it, as is. And for weights which survive the pruning,
        # reinitialize it to the value, the model received BEFORE it was trained and pruned-
        for orig_wts, pruned_wts in zip(orig_model_stripped.trainable_weights,
                                        winning_ticket_model_stripped.trainable_weights):
            pruned_wts.assign(tf.where(tf.equal(pruned_wts, 0), pruned_wts, orig_wts))